In [1]:
import os
import numpy as np
from collections import OrderedDict

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
from tflite import Model

data = open("./mediapipe/mediapipe/models/face_landmark.tflite", "rb").read()
model = Model.Model.GetRootAsModel(data, 0)

In [3]:
subgraph = model.Subgraphs(0)
subgraph.Name()

b'facemesh_facecontours_faceflag-blaze_shift30-2019_01_14-v0.tflite.no_meta'

In [4]:
def get_shape(tensor):
    return [tensor.Shape(i) for i in range(tensor.ShapeLength())]

In [5]:
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    print("%3d %30s %d %2d %s" % (i, tensor.Name(), tensor.Type(), tensor.Buffer(), 
                                  get_shape(subgraph.Tensors(i))))

  0                     b'input_1' 0  0 [1, 192, 192, 3]
  1               b'conv2d/Kernel' 0  1 [16, 3, 3, 3]
  2                 b'conv2d/Bias' 0  2 [16]
  3                      b'conv2d' 0  0 [1, 96, 96, 16]
  4               b'p_re_lu/Alpha' 0  3 [1, 1, 16]
  5                     b'p_re_lu' 0  0 [1, 96, 96, 16]
  6     b'depthwise_conv2d/Kernel' 0  4 [1, 3, 3, 16]
  7       b'depthwise_conv2d/Bias' 0  5 [16]
  8            b'depthwise_conv2d' 0  0 [1, 96, 96, 16]
  9             b'conv2d_1/Kernel' 0  6 [16, 1, 1, 16]
 10               b'conv2d_1/Bias' 0  7 [16]
 11                    b'conv2d_1' 0  0 [1, 96, 96, 16]
 12                         b'add' 0  0 [1, 96, 96, 16]
 13             b'p_re_lu_1/Alpha' 0  8 [1, 1, 16]
 14                   b'p_re_lu_1' 0  0 [1, 96, 96, 16]
 15   b'depthwise_conv2d_1/Kernel' 0  9 [1, 3, 3, 16]
 16     b'depthwise_conv2d_1/Bias' 0 10 [16]
 17          b'depthwise_conv2d_1' 0  0 [1, 96, 96, 16]
 18             b'conv2d_2/Kernel' 0 11 [16, 1, 1, 1

In [6]:
tensor_dict = {(subgraph.Tensors(i).Name().decode("utf8")): i 
               for i in range(subgraph.TensorsLength())}

In [7]:
parameters = {}
for i in range(subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0:
        name = tensor.Name().decode("utf8")
        parameters[name] = tensor.Buffer()

len(parameters)

116

In [8]:
def get_weights(tensor_name):
    i = tensor_dict[tensor_name]
    tensor = subgraph.Tensors(i)
    buffer = tensor.Buffer()
    shape = get_shape(tensor)
    assert(tensor.Type() == 0)  # FLOAT32
    
    W = model.Buffers(buffer).DataAsNumpy()
    W = W.view(dtype=np.float32)
    W = W.reshape(shape)
    return W

In [9]:
W = get_weights("conv2d/Kernel")
b = get_weights("conv2d/Bias")
W.shape, b.shape

((16, 3, 3, 3), (16,))

In [10]:
import torch
from blazeFacemark import BlazeFacemark

In [11]:
net = BlazeFacemark()
net

BlazeFacemark(
  (backbone1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): PReLU(num_parameters=16)
    (2): BlazeBlock(
      (convs): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
        (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (act): PReLU(num_parameters=16)
    )
    (3): BlazeBlock(
      (convs): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
        (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (act): PReLU(num_parameters=16)
    )
    (4): BlazeBlock(
      (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (convs): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), groups=16)
        (1): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (act): PReLU(num_parameters=32)
    )
    (5): BlazeBlock(
      (co

In [12]:
probable_names = []
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0 and tensor.Type() == 0:
        print(tensor.Name())
        probable_names.append(tensor.Name().decode("utf-8"))
        
        

b'conv2d/Kernel'
b'conv2d/Bias'
b'p_re_lu/Alpha'
b'depthwise_conv2d/Kernel'
b'depthwise_conv2d/Bias'
b'conv2d_1/Kernel'
b'conv2d_1/Bias'
b'p_re_lu_1/Alpha'
b'depthwise_conv2d_1/Kernel'
b'depthwise_conv2d_1/Bias'
b'conv2d_2/Kernel'
b'conv2d_2/Bias'
b'p_re_lu_2/Alpha'
b'depthwise_conv2d_2/Kernel'
b'depthwise_conv2d_2/Bias'
b'conv2d_3/Kernel'
b'conv2d_3/Bias'
b'p_re_lu_3/Alpha'
b'depthwise_conv2d_3/Kernel'
b'depthwise_conv2d_3/Bias'
b'conv2d_4/Kernel'
b'conv2d_4/Bias'
b'p_re_lu_4/Alpha'
b'depthwise_conv2d_4/Kernel'
b'depthwise_conv2d_4/Bias'
b'conv2d_5/Kernel'
b'conv2d_5/Bias'
b'p_re_lu_5/Alpha'
b'depthwise_conv2d_5/Kernel'
b'depthwise_conv2d_5/Bias'
b'conv2d_6/Kernel'
b'conv2d_6/Bias'
b'p_re_lu_6/Alpha'
b'depthwise_conv2d_6/Kernel'
b'depthwise_conv2d_6/Bias'
b'conv2d_7/Kernel'
b'conv2d_7/Bias'
b'p_re_lu_7/Alpha'
b'depthwise_conv2d_7/Kernel'
b'depthwise_conv2d_7/Bias'
b'conv2d_8/Kernel'
b'conv2d_8/Bias'
b'p_re_lu_8/Alpha'
b'depthwise_conv2d_8/Kernel'
b'depthwise_conv2d_8/Bias'
b'conv2d_9/

In [13]:
convert = {}
i = 0
for name, params in net.state_dict().items():
    convert[name] = probable_names[i]
    i += 1

In [17]:
new_state_dict = OrderedDict()

for dst, src in convert.items():
    W = get_weights(src)
    print(dst, src, W.shape, net.state_dict()[dst].shape)

    if W.ndim == 4:       
        if W.shape[0] == 1 and src != 'conv2d_30/Kernel':
            W = W.transpose((3, 0, 1, 2))  # depthwise conv
        else:
            W = W.transpose((0, 3, 1, 2))  # regular conv        
            
    if W.ndim == 3:
        W = W.reshape(W.shape[2]) #PReLU 
        
    
    
    new_state_dict[dst] = torch.from_numpy(W)

backbone1.0.weight conv2d/Kernel (16, 3, 3, 3) torch.Size([16, 3, 3, 3])
backbone1.0.bias conv2d/Bias (16,) torch.Size([16])
backbone1.1.weight p_re_lu/Alpha (1, 1, 16) torch.Size([16])
backbone1.2.convs.0.weight depthwise_conv2d/Kernel (1, 3, 3, 16) torch.Size([16, 1, 3, 3])
backbone1.2.convs.0.bias depthwise_conv2d/Bias (16,) torch.Size([16])
backbone1.2.convs.1.weight conv2d_1/Kernel (16, 1, 1, 16) torch.Size([16, 16, 1, 1])
backbone1.2.convs.1.bias conv2d_1/Bias (16,) torch.Size([16])
backbone1.2.act.weight p_re_lu_1/Alpha (1, 1, 16) torch.Size([16])
backbone1.3.convs.0.weight depthwise_conv2d_1/Kernel (1, 3, 3, 16) torch.Size([16, 1, 3, 3])
backbone1.3.convs.0.bias depthwise_conv2d_1/Bias (16,) torch.Size([16])
backbone1.3.convs.1.weight conv2d_2/Kernel (16, 1, 1, 16) torch.Size([16, 16, 1, 1])
backbone1.3.convs.1.bias conv2d_2/Bias (16,) torch.Size([16])
backbone1.3.act.weight p_re_lu_2/Alpha (1, 1, 16) torch.Size([16])
backbone1.4.convs.0.weight depthwise_conv2d_2/Kernel (1, 3, 

In [18]:
net.load_state_dict(new_state_dict, strict=True)

<All keys matched successfully>

In [19]:
torch.save(net.state_dict(), "blazeFacemark.pth")